# Advent of Code 2022, Day 25: SNAFU numbers

In [280]:
#= 

Step 1: Read the input, store it in 'lines'

=#

filename = "input24.txt"


f = open(filename, "r")
lines = readlines(f)
close(f)

In [281]:
#= 

Step 2: Convert SNAFU to decimal and vice-versa. 

=#

function SNAFUtoDecimal(string::String)
    SNAFUDict = Dict('0' => 0, '1' => 1, '2' => 2, '-' => -1, '=' => -2)

    number = 0::Int
    len = length(string)
    for i=0:len-1
        digit = SNAFUDict[string[end-i]]
        number = number + 5^i * digit
    end 
    return number
end

#= 

We read all the input numbers and add them together. 

=#

counter = 0
for l in 1:length(lines)
    counter += SNAFUtoDecimal(lines[l])
end

# We find that the sum of all the given SNAFU numbers is: 
println(counter)

36251175625102


In [284]:
#= 

Step 3: convert the sum back to SNAFU. 

Now, actually most code is needed to turn this (decimal) SUM back into a SNAFU number. We do this in 3 steps: 

1. determine the number of snafu digits
2. rewrite the number in 5-ary  
3. Take the 5-ary number, and from the least significant digit onwards, 
    follor ReplacementRules (3 -> =, 4 -> -, 5-> 0), while keeping track of a potential carry.


=#

# Write a decimal number in 5-ary. 
function decimalToFiveary( decimal )

    # Determine the number of SNAFU digits needed. 
    num_digits = ceil( Int, log(decimal)/log(5) ) + 2 # With built-in safety for overflow. 

    # Create empty array of given length
    digitlist = fill(0, num_digits) 

    # Fill the 5-ary digits:
    for j in 1:num_digits
        basenum = 5^(num_digits - j)
        digitlist[j] = floor( Int, decimal / basenum )
        decimal = decimal - digitlist[j] * basenum  
    end

    return digitlist
end

# Replacement rules for five-ary number to SNAFU.
# When replacing a number, we account for a CARRY (to increment the next most significant number)
# Return value is [Carry, SNAFU digit]
function replacementRules( digit )
    if digit == 5
        return [1, '0']
    elseif digit == 4 
        return [1, '-']
    elseif digit == 3
        return [1, '=']
    elseif digit > 5 || digit < 0  # Value of 5 should be accepted (due to carry), higher than 5 shouldn't occur. 
        throw(DomainError("Error in function replacementRules: digit not in range 0..5"))
    else 
        return [0, Char('0' + digit)]  # The latter creates a 'char' of the given number. 
    end
end


# Turn a five-ary number into a SNAFU number. 
function fivearyToSNAFU( digitlist )
    len = length(digitlist)
    SNAFUlist = Array{Any}(nothing, len+1) # Make an array that can hold anything
    SNAFUlist[1] = 0
    SNAFUlist[2:end] =  digitlist 

    # For all digits (running right-to-left), perform the substitution
    for j in len+1:-1:2
        repl = replacementRules( SNAFUlist[j] )
        SNAFUlist[j] = repl[2]
        SNAFUlist[j-1] += repl[1] 
    end
    
    # Special treatment of most significant digit: turn into char.
    SNAFUlist[1] = replacementRules(SNAFUlist[1])[2]

    return SNAFUlist
end

# Turn a decimal number into a SNAFU number. 
function decimalToSNAFU( decimal )
    fiveary = decimalToFiveary( decimal )
    return join( fivearyToSNAFU( fiveary ) ) # Join chars together to form a proper String. 
end


#= 

Okay, let's see what SNAFU number we get?

=# 
println( decimalToSNAFU( counter ))

# We can re-convert this number back into decimal, to convince ourselves that no mistakes were made. 
println( SNAFUtoDecimal("20===-20-020=0001-02"))


00020===-20-020=0001-02
36251175625102
